## PyTorch in Production


https://classroom.udacity.com/courses/ud188/lessons/d45cb502-8355-4f5f-a4b2-a6d1b4294fcc/concepts/87f3d234-1d84-45fb-b507-eec5bfedf860


Welcome to this lesson on using PyTorch in production. PyTorch has been most popular in research settings due to its flexibility, expressiveness, and ease of development in general. However, adoption has been slow in industry because it wasn't as useful in production environments which typically require models to run in C++. To address this, PyTorch 1.0 has introduced new features for exporting your models from Python into C++.

I'll be showing you how to use these new features to compile Python models for use in C++ environments. Follow along while I walk through this tutorial.
https://pytorch.org/tutorials/advanced/cpp_export.html
I'll show you how to convert your models into a new representation called Torch Script.
https://pytorch.org/docs/master/jit.html
Using two methods, tracing and annotations, you can convert your models into ScriptModules which can be exported as serialized files then loaded into a C++ application. Using this combination of Torch Script and the PyTorch C++ API, you can do all the development and training of your network in Python, then utilize the trained model in a C++ application.

First check PyTorch version, if above 1.0 all good
look fine 

In [4]:
import torch
print(torch.__version__)

1.6.0


##### Step 1 converting your pytorch model to torch script

#####   via tracing

To convert PyTorch model to TorchScript via tracing you must pass an instance of your model along with an example input to the torch.jit.trace function.
This will produce a torch.jit.ScriptModule object with the trace of your model evaluation embedded in the module's forward method:

In [5]:
import torch
import torchvision

#an instance of your model
#just use resnet18 model, but could be any model you trained
model = torchvision.models.resnet18()

#an exmaple input you would normally provide to your model's forward() method
#should be same shape as in a model (this one is same as resnet18)
example = torch.rand(1,3,224,224)

#use torch.jit.trace to generate a torch.jit.ScriptModule visa tracing
traced_script_module = torch.jit.trace(model, example)



In [6]:
output = traced_script_module(torch.ones(1,3,224,224))

In [7]:
output[0, :5]

tensor([ 0.3728, -0.6737,  0.1446,  0.3442, -0.2913], grad_fn=<SliceBackward>)

###### Converting to Torch Script via Annotation
Under certain circumstances, such as if your model employs particular forms of control flow, you may want to write your model in Torch Script directly and annotate your model accordingly. For example, say you have the following vanilla Pytorch model:

In [ ]:
import torch

class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

Because the forward method of this module uses control flow that is dependent on the input, it is not suitable for tracing. Instead, we can convert it to a ScriptModule. In order to convert the module to the ScriptModule, one needs to compile the module with torch.jit.script as follows:

In [ ]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

If you need to exclude some methods in your nn.Module because they use Python features that TorchScript doesn’t support yet, you could annotate those with @torch.jit.ignore

my_module is an instance of ScriptModule that is ready for serialization.

#### Step 2: Serializing Your Script Module to a File
Once you have a ScriptModule in your hands, either from tracing or annotating a PyTorch model, you are ready to serialize it to a file. Later on, you’ll be able to load the module from this file in C++ and execute it without any dependency on Python. Say we want to serialize the ResNet18 model shown earlier in the tracing example. To perform this serialization, simply call save on the module and pass it a filename:

This will produce a traced_resnet_model.pt file in your working directory. If you also would like to serialize my_module, call my_module.save("my_module_model.pt") We have now officially left the realm of Python and are ready to cross over to the sphere of C++.


In [ ]:
traced_script_module.save("traced_resnet_model.pt")


#### Step 3: Loading Your Script Module in C++
To load your serialized PyTorch model in C++, your application must depend on the PyTorch C++ API – also known as LibTorch. The LibTorch distribution encompasses a collection of shared libraries, header files and CMake build configuration files. While CMake is not a requirement for depending on LibTorch, it is the recommended approach and will be well supported into the future. For this tutorial, we will be building a minimal C++ application using CMake and LibTorch that simply loads and executes a serialized PyTorch model.

https://pytorch.org/tutorials/advanced/cpp_export.html

https://medium.com/search?q=lstm


next 
http://localhost:8888/notebooks/Documents/olga_data_science_machine_learning2/99_Keras_LSTM_gender_prediction.ipynb

after
https://classroom.udacity.com/courses/ud190